In [12]:
%matplotlib inline

from scipy.odr import *
from scipy.stats import *
import numpy as np
import pandas as pd
import os
import time
import matplotlib.pyplot as plt
import ast
from multiprocessing import Pool, cpu_count

import scipy

from IPython import display
from matplotlib.patches import Rectangle

from sklearn.metrics import mean_squared_error
import json

import scipy.stats as st
from sklearn.metrics import r2_score


from matplotlib import cm
from mpl_toolkits.mplot3d import axes3d
import matplotlib.pyplot as plt

import copy

from sklearn.model_selection import LeaveOneOut, LeavePOut

from multiprocessing import Pool
import cv2

In [4]:
df_filtered = pd.read_pickle("PklData/dfFiltered_LSTM.pkl")

In [5]:
df_filtered.head()

,userID,Timestamp,Current_Task,Task_amount,TaskID,VersionID,RepetitionID,Actual_Data,Is_Pause,Image
5511,2,1545154138673,1,680,20,2,1,True,False,"[0, 0, 1, 3, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, ..."
5512,2,1545154138713,1,680,20,2,1,True,False,"[0, 0, 1, 3, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, ..."
5513,2,1545154138755,1,680,20,2,1,True,False,"[0, 0, 1, 3, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, ..."
5514,2,1545154138803,1,680,20,2,1,True,False,"[0, 0, 1, 3, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, ..."
5515,2,1545154138846,1,680,20,2,1,True,False,"[0, 0, 1, 3, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, ..."


In [6]:
df_filtered.Image = df_filtered.Image.apply(lambda x: x.reshape(27, 15))
df_filtered.Image = df_filtered.Image.apply(lambda x: x.clip(min=0, max=255))
df_filtered.Image = df_filtered.Image.apply(lambda x: x.astype(np.uint8))
df_filtered["ImageSum"] = df_filtered.Image.apply(lambda x: np.sum(x))

In [10]:
#LSTMs new Blob detection
def detect_blobs(image):
    #image = image.reshape(27, 15)
    large = np.ones((29,17), dtype=np.uint8)
    large[1:28,1:16] = image
    temp, thresh = cv2.threshold(cv2.bitwise_not(large), 200, 255, cv2.THRESH_BINARY)
    im2, contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    contours = [a for a in contours if cv2.contourArea(a) > 5 and cv2.contourArea(a) < 255]
    lstBlob  = []
    lstMin = []
    lstMax = []
    count = 0
    return len(contours) > 0

In [15]:
%%time
pool = Pool(cpu_count() - 2)
temp_blobs = pool.map(detect_blobs, df_filtered.Image)
pool.close()

CPU times: user 1.79 s, sys: 574 ms, total: 2.36 s
Wall time: 2.51 s


In [16]:
df_filtered["ContainsBlobs"] = temp_blobs

In [17]:
#Label if knuckle or finger
def f(row):
    if row['TaskID'] < 17:
        #val = "Knuckle"
        val = 0
    elif row['TaskID'] >= 17:
        #val = "Finger"
        val = 1
    return val
df_filtered['InputMethod'] = df_filtered.apply(f, axis=1)

In [26]:
df_filtered.columns

Index(['userID', 'Timestamp', 'Current_Task', 'Task_amount', 'TaskID',
       'VersionID', 'RepetitionID', 'Actual_Data', 'Is_Pause', 'Image',
       'ImageSum', 'ContainsBlobs', 'InputMethod'],
      dtype='object')

In [28]:
%%time
for userID in df_filtered.userID.unique():
    for TaskID in df_filtered[df_filtered.userID == userID].TaskID.unique():
        for VersionID in df_filtered[(df_filtered.userID == userID) & (df_filtered.TaskID == TaskID)].VersionID.unique():
            first_blob = 0
            last_blob = 0
            for index, row in df_filtered[(df_filtered.userID == userID) & (df_filtered.TaskID == TaskID) & (df_filtered.VersionID == VersionID)].iterrows():
                if row.ContainsBlobs:
                    last_blob = row.Timestamp
                    if first_blob == 0:
                        first_blob = row.Timestamp

1,54515E+12 to 1,54515E+12


In [18]:
df_filtered.columns

Index(['userID', 'Timestamp', 'Current_Task', 'Task_amount', 'TaskID',
       'VersionID', 'RepetitionID', 'Actual_Data', 'Is_Pause', 'Image',
       'ImageSum', 'ContainsBlobs', 'InputMethod'],
      dtype='object')

In [ ]:
df[["userID", "TaskID", "Version", "Blobs", "InputMethod"]].to_pickle("PklData/df_lstm.pkl")